In [109]:
import pandas as pd

In [110]:
pd.set_option('display.max_columns', None)
store_records = pd.read_csv('../data/sample_superstore_updated.csv')
store_records['order_year'] = store_records['order_date'].str[-4:]
store_records.head()

,row_id,order_id,order_date,ship_date,ship_mode,customer_id,customer_name,segment,country,city,state,postal_code,region,product_id,category,sub_category,product_name,sales,quantity,discount,profit,order_year
0,1,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136,2016
1,2,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820,2016
2,3,CA-2016-138688,6/12/2016,6/16/2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714,2016
3,4,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310,2015
4,5,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164,2015


## KPI Data Points for Dashboard

In [111]:
# category of most sold products (rank top 3)
# there are only 3 main categories, but this is where each ranks

top_3_category = store_records[['product_id', 'quantity', 'product_name','category','sub_category']].groupby('category').agg({'product_id':'count','quantity':'sum'}).rename(columns={'product_id':'product_id_count'}).sort_values(by='quantity', ascending=False).reset_index()
top_3_category

,category,product_id_count,quantity
0,Office Supplies,6026,22906
1,Furniture,2121,8028
2,Technology,1847,6939


In [112]:
# category of most sold products (rank top 5)

top_5_sub_category = store_records[['product_id', 'quantity', 'product_name','category','sub_category']].groupby('sub_category').agg({'product_id':'count', 'quantity':'sum'}).rename(columns={'product_id':'product_id_count'}).sort_values(by='quantity', ascending=False).reset_index()
top_5_sub_category = top_5_sub_category.iloc[0:5]
top_5_sub_category

,sub_category,product_id_count,quantity
0,Binders,1523,5974
1,Paper,1370,5178
2,Furnishings,957,3563
3,Phones,889,3289
4,Storage,846,3158


In [113]:
# top 5 list of products that generate the most profit

high_profit_products = store_records[['product_id', 'product_name','category','sub_category','profit']].groupby('product_id').agg({'profit':'sum', 'product_name':'first', 'category':'first', 'sub_category':'first'}).sort_values(by='profit', ascending=False).reset_index()
high_profit_products_top_5 = high_profit_products.iloc[0:5]
high_profit_products_top_5

,product_id,profit,product_name,category,sub_category
0,TEC-CO-10004722,25199.9280,Canon imageCLASS 2200 Advanced Copier,Technology,Copiers
1,OFF-BI-10003527,7753.0390,Fellowes PB500 Electric Punch Plastic Comb Bin...,Office Supplies,Binders
2,TEC-CO-10001449,6983.8836,Hewlett Packard LaserJet 3310 Copier,Technology,Copiers
3,TEC-CO-10003763,4570.9347,Canon PC1060 Personal Laser Copier,Technology,Copiers
4,TEC-AC-10002049,4425.3432,Logitech G19 Programmable Gaming Keyboard,Technology,Accessories


In [114]:
# top 5 list of products that generate the least profit

neg_profit_products = store_records[['product_id', 'product_name','category','sub_category','profit']].groupby('product_id').agg({'profit':'sum', 'product_name':'first', 'category':'first', 'sub_category':'first'}).sort_values(by='profit', ascending=True).reset_index()
neg_profit_products_top_5 = neg_profit_products.iloc[0:5]
neg_profit_products_top_5

,product_id,profit,product_name,category,sub_category
0,TEC-MA-10000418,-8879.9704,Cubify CubeX 3D Printer Double Head Print,Technology,Machines
1,TEC-MA-10000822,-4589.9730,Lexmark MX611dhe Monochrome Laser Printer,Technology,Machines
2,TEC-MA-10004125,-3839.9904,Cubify CubeX 3D Printer Triple Head Print,Technology,Machines
3,FUR-TA-10000198,-2876.1156,Chromcraft Bull-Nose Wood Oval Conference Tabl...,Furniture,Tables
4,FUR-TA-10001889,-1934.3976,Bush Advantage Collection Racetrack Conference...,Furniture,Tables


In [115]:
# states that place the most orders

high_business_states = store_records[['product_id','quantity','state','region']].groupby('state').agg({'product_id':'count'}).sort_values(by='product_id', ascending=False).reset_index()
high_business_states = high_business_states.iloc[0:5]
high_business_states

,state,product_id
0,California,2001
1,New York,1128
2,Texas,985
3,Pennsylvania,587
4,Washington,506


In [116]:
# regions that place the most orders

high_business_states = store_records[['product_id','state','region']].groupby('region').agg({'product_id':'count'}).sort_values(by='product_id', ascending=False).reset_index()
high_business_states = high_business_states.iloc[0:4]
high_business_states

,region,product_id
0,West,3203
1,East,2848
2,Central,2323
3,South,1620


In [117]:
# popular orders by region

popular_by_region = store_records[['product_id','quantity','category','sub_category','region']].groupby(['region','category']).agg({'product_id':'count', 'quantity':'sum'}).sort_values(by='product_id', ascending=False).reset_index()
popular_by_region = popular_by_region.rename(columns={'product_id':'product_id_count'})
popular_by_region = popular_by_region.iloc[0:5]
popular_by_region

,region,category,product_id_count,quantity
0,West,Office Supplies,1897,7235
1,East,Office Supplies,1712,6462
2,Central,Office Supplies,1422,5409
3,South,Office Supplies,995,3800
4,West,Furniture,707,2696


In [118]:
# states that purchase the most products 

high_business_states_q = store_records[['product_id','quantity','state','region']].groupby('state').agg({'quantity':'sum'}).sort_values(by='quantity', ascending=False).reset_index()
high_business_states_q = high_business_states_q.iloc[0:5]
high_business_states_q

,state,quantity
0,California,7667
1,New York,4224
2,Texas,3724
3,Pennsylvania,2153
4,Washington,1883


In [119]:
# regions that purchase the most products 

high_business_states = store_records[['product_id','quantity','state','region']].groupby('region').agg({'product_id':'count', 'quantity':'sum'}).sort_values(by='product_id', ascending=False).reset_index()
high_business_states = high_business_states.rename(columns={'product_id':'product_id_count'})
high_business_states = high_business_states.iloc[0:4]
high_business_states

,region,product_id_count,quantity
0,West,3203,12266
1,East,2848,10618
2,Central,2323,8780
3,South,1620,6209


In [123]:
popular_products_region = store_records[['product_id','quantity','state','region','product_name']].groupby(['product_id','region']).agg({'quantity':'sum', 'product_name':'first'}).sort_values(by='quantity', ascending=False).reset_index()
popular_products_region

,product_id,region,quantity,product_name
0,TEC-AC-10003832,West,45,Logitech P710e Mobile Speakerphone
1,OFF-BI-10000301,Central,34,GBC Instant Report Kit
2,OFF-BI-10000756,Central,33,Storex DuraTech Recycled Plastic Frosted Binders
3,OFF-PA-10001970,East,33,Xerox 1881
4,OFF-BI-10003656,East,32,Fellowes PB200 Plastic Comb Binding Machine
...,...,...,...,...
5293,OFF-PA-10000048,West,1,Xerox 20
5294,OFF-BI-10000309,West,1,"GBC Twin Loop Wire Binding Elements, 9/16"" Spi..."
5295,OFF-BI-10000315,Central,1,Poly Designer Cover & Back
5296,OFF-BI-10004738,West,1,Flexible Leather- Look Classic Collection Ring...


In [125]:
# popular products in the west
popular_products_region[popular_products_region['region'] == 'West'].head(5)

,product_id,region,quantity,product_name
0,TEC-AC-10003832,West,45,Logitech P710e Mobile Speakerphone
5,OFF-BI-10000174,West,32,Wilson Jones Clip & Carry Folder Binder Tool f...
8,OFF-BI-10001036,West,31,Cardinal EasyOpen D-Ring Binders
13,OFF-BI-10001670,West,29,Vinyl Sectional Post Binders
14,OFF-ST-10002486,West,29,Eldon Shelf Savers Cubes and Bins


In [130]:
# profits by shipping type

high_profit_shipping = store_records[['product_id','ship_mode','profit']].groupby('ship_mode').agg({'profit':'sum', 'product_id':'count'}).sort_values(by='profit', ascending=False).reset_index()
high_profit_shipping = high_profit_shipping.rename(columns={'order_id':'order_count'})
high_profit_shipping

,ship_mode,profit,product_id
0,Standard Class,164088.7875,5968
1,Second Class,57446.6354,1945
2,First Class,48969.8399,1538
3,Same Day,15891.7589,543
